In [12]:
#dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date


In [13]:
#import data
measurements = pd.read_csv('clean_hawaii_measurements.csv')
stations = pd.read_csv('clean_hawaii_stations.csv')
measurements['date'] = pd.to_datetime(measurements['date'], format='%Y-%m-%d')
measurements.head()

,Unnamed: 0,station,date,prcp,tobs
0,0,USC00519397,2010-01-01,0.08,65
1,1,USC00519397,2010-01-02,0.00,63
2,2,USC00519397,2010-01-03,0.00,74
3,3,USC00519397,2010-01-04,0.00,76
4,4,USC00519397,2010-01-06,0.00,73


In [14]:
#convert to dict
measurements_data = measurements.to_dict(orient='records')
measurements_data[0]

{'Unnamed: 0': 0,
 'date': Timestamp('2010-01-01 00:00:00'),
 'prcp': 0.08,
 'station': 'USC00519397',
 'tobs': 65}

In [15]:
#create engine
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

conn = engine.connect() 

In [16]:
#create classes
Base = declarative_base()


class Station(Base):
    __tablename__ = 'station'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    name = Column(String(255))
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
class Measurement(Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(String(255))
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)
    
    
Base.metadata.create_all(engine)

In [17]:
#add tables
metadata = MetaData(bind=engine)
metadata.reflect()

measurement_table = sqlalchemy.Table('measurements', metadata, autoload=True)
#conn.execute(measurement_table.delete())

conn.execute(measurement_table.insert(), measurements_data)


OperationalError: (sqlite3.OperationalError) database is locked (Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
#test insert
conn.execute('select * from measurements limit 5').fetchall()

In [ ]:
#insert station table
station_data = stations.to_dict(orient='records')

station_data[0]

In [ ]:
station_table = sqlalchemy.Table('station', metadata, autoload=True)

conn.execute(station_table.delete())

conn.execute(station_table.insert(), station_data)

conn.execute('select * from station limit 5').fetchall()